# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import numpy as np
import datetime

### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [15]:
today = datetime.date.today()
date_range = np.arange('2015-01-01', today, dtype='datetime64[D]')
date_range

array(['2015-01-01', '2015-01-02', '2015-01-03', ..., '2025-09-24',
       '2025-09-25', '2025-09-26'], shape=(3922,), dtype='datetime64[D]')

### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

In [16]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("ALPHA_API_KEY")

print(bool(api_key))

True


### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [17]:
df_apple = web.DataReader('AAPL', 'stooq', date_range[0], date_range[-1])
df_apple.head()

,Open,High,Low,Close,Volume
Date,,,,,
2025-09-26,254.095,257.60,253.780,255.46,45571892
2025-09-25,253.205,257.17,251.712,256.87,55202075
2025-09-24,255.220,255.74,251.040,252.31,42303710
2025-09-23,255.875,257.34,253.580,254.43,60275187
2025-09-22,248.300,256.64,248.120,256.08,105517416


### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [18]:
df_apple['stock'] = 'AAPL'

### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [19]:
df_tesla = web.DataReader('TSLA', 'stooq', date_range[0], date_range[-1])
df_tesla['stock'] = 'TSLA'

df_ibm = web.DataReader('IBM', 'stooq', date_range[0], date_range[-1])
df_ibm['stock'] = 'IBM'

df_msft = web.DataReader('MSFT', 'stooq', date_range[0], date_range[-1])
df_msft['stock'] = 'MSFT'

### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [20]:
df = pd.concat([df_apple, df_tesla, df_ibm, df_msft])
df.stock.unique()

array(['AAPL', 'TSLA', 'IBM', 'MSFT'], dtype=object)

### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [21]:
df.reset_index(inplace=True)

df.set_index(['stock', 'Date'], inplace=True)

df.head()

Open    High      Low   Close       Volume
stock Date                                                     
AAPL  2025-09-26  254.095  257.60  253.780  255.46   45571892.0
      2025-09-25  253.205  257.17  251.712  256.87   55202075.0
      2025-09-24  255.220  255.74  251.040  252.31   42303710.0
      2025-09-23  255.875  257.34  253.580  254.43   60275187.0
      2025-09-22  248.300  256.64  248.120  256.08  105517416.0

### Step 7. Create a dataFrame called vol, with the volume values.

In [22]:
vol = df['Volume']
vol.head()

stock  Date      
AAPL   2025-09-26     45571892.0
       2025-09-25     55202075.0
       2025-09-24     42303710.0
       2025-09-23     60275187.0
       2025-09-22    105517416.0
Name: Volume, dtype: float64

### Step 8. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [24]:
weekly = vol.resample('W', level='Date').sum()
weekly.head()

Date
2015-01-04    3.509238e+08
2015-01-11    1.832776e+09
2015-01-18    2.043929e+09
2015-01-25    1.695309e+09
2015-02-01    2.891617e+09
Freq: W-SUN, Name: Volume, dtype: float64

### Step 9. Find all the volume traded in the year of 2015

In [27]:
vol_2015 = vol[vol.index.get_level_values('Date').year == 2015]

vol_2015.head()

stock  Date      
AAPL   2015-12-31    171629747.0
       2015-12-30     98331163.0
       2015-12-29    136718750.0
       2015-12-28    118034904.0
       2015-12-24     60098492.0
Name: Volume, dtype: float64